## Assignment 4: Obsevational Studies and Applied ML

### Deadline
November 21st,11:59PM

### Important notes

Make sure you push on GitHub your notebook with all the cells already evaluated. Don't forget to add a textual description of your thought process, the assumptions you made, and the solution you implemented. Back up any hypotheses and claims with data, since this is an important aspect of the course. Please write all your comments in English, and use meaningful variable names in your code. Your repo should have a single notebook (plus the data files necessary) in the master branch. If there are multiple notebooks present, we will not grade anything.

Use this legendary link to create your repository: [link](https://classroom.github.com/g/YXtsr0QK)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.preprocessing import OneHotEncoder
from pandas.plotting import scatter_matrix
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_val_score
import seaborn as sns

%matplotlib inline

In [2]:
data_folder = './data/'

## Task 1: Boosting the economy by incentivizing self-employment

Assume the biggest priority of the local government in 2018 is to increase per-capita income. To do so, the officials plan to adopt a strategy for incentivizing self-employment through a series of campaigns, educational programs, and dedicated funds.

Since it is unethical and impossible in this setting to run a controlled experiment involving citizens (e.g., fire employees and force them to self-employ), the officials have asked you, the data scientist, to establish the effect of self-employment on the economy, relying on observational data.

**A)** You will be working with the full US 2015 census dataset (acs2015_county_data.csv, available at https://www.kaggle.com/muonneutrino/us-census-demographic-data#acs2015_county_data.csv). Using suitable methods, determine and quantify the dependency between the percentage of self-employed citizens and per capita income across all 3,212 US counties. Do citizens in counties that have a higher percentage of self-employed people earn more per capita?

**B)** The pilot program will involve all counties within a limited set of three US states. Set A includes Wisconsin, Tennessee, and  Minnesota. Quantify the dependency of per-capita income on self-employment rates across all the counties in set A.

**C)** In which state within set A is the observed effect of self-employment on per-capita income the strongest?

**D)** Set B includes New Jersey, Kansas, and Rhode Island. Repeat the analysis from steps B and C above, but now for set B. In which of the two sets A and B (if any) would you recommend incentivizing self-employment? Explain your reasoning.

Hint: It is useful to add a notion of confidence to your results and explore the data visually. You are allowed to use the SciPy library.

In [8]:
pd.set_option('display.max_columns', None)

In [9]:
data = pd.read_csv(data_folder+'acs2015_county_data.csv', index_col=0)
data.head()

State   County  TotalPop    Men  Women  Hispanic  White  Black  \
CensusId                                                                     
1001      Alabama  Autauga     55221  26745  28476       2.6   75.8   18.5   
1003      Alabama  Baldwin    195121  95314  99807       4.5   83.1    9.5   
1005      Alabama  Barbour     26932  14497  12435       4.6   46.2   46.7   
1007      Alabama     Bibb     22604  12073  10531       2.2   74.5   21.4   
1009      Alabama   Blount     57710  28512  29198       8.6   87.9    1.5   

          Native  Asian  Pacific  Citizen   Income  IncomeErr  IncomePerCap  \
CensusId                                                                      
1001         0.4    1.0      0.0    40725  51281.0     2391.0         24974   
1003         0.6    0.7      0.0   147695  50254.0     1263.0         27317   
1005         0.2    0.4      0.0    20714  32964.0     2973.0         16824   
1007         0.4    0.1      0.0    17495  38678.0     3995.0         18431   
1009         0.3    0.1      0.0    42345  45813.0     3141.0         20532   

          IncomePerCapErr  Poverty  ChildPoverty  Professional  Service  \
CensusId                                                                  
1001                 1080     12.9          18.6          33.2     17.0   
1003                  711     13.4          19.2          33.1     17.7   
1005                  798     26.7          45.3          26.8     16.1   
1007                 1618     16.8          27.9          21.5     17.9   
1009                  708     16.7          27.2          28.5     14.1   

          Office  Construction  Production  Drive  Carpool  Transit  Walk  \
CensusId                                                                    
1001        24.2           8.6        17.1   87.5      8.8      0.1   0.5   
1003        27.1          10.8        11.2   84.7      8.8      0.1   1.0   
1005        23.1          10.8        23.1   83.8     10.9      0.4   1.8   
1007        17.8          19.0        23.7   83.2     13.5      0.5   0.6   
1009        23.9          13.5        19.9   84.9     11.2      0.4   0.9   

          OtherTransp  WorkAtHome  MeanCommute  Employed  PrivateWork  \
CensusId                                                                
1001              1.3         1.8         26.5     23986         73.6   
1003              1.4         3.9         26.4     85953         81.5   
1005              1.5         1.6         24.1      8597         71.8   
1007              1.5         0.7         28.8      8294         76.8   
1009              0.4         2.3         34.9     22189         82.0   

          PublicWork  SelfEmployed  FamilyWork  Unemployment  
CensusId                                                      
1001            20.9           5.5         0.0           7.6  
1003            12.3           5.8         0.4           7.5  
1005            20.8           7.3         0.1          17.6  
1007            16.1           6.7         0.4           8.3  
1009            13.5           4.2         0.4           7.7

### 1.A 

In [14]:
fig =  data.plot(kind = 'scatter', x = 'IncomePerCap', y = 'SelfEmployed')

In [15]:
data[['IncomePerCap','SelfEmployed']].corr()

IncomePerCap  SelfEmployed
IncomePerCap      1.000000      0.087274
SelfEmployed      0.087274      1.000000

Very low correlation is present between the two variables.

### 1.B

In [20]:
setA = ['Winsconsin','Minnesota','Tennessee']
mask = data['State'].isin(setA)
dataA = data[mask]
dataA.head()

State    County  TotalPop     Men   Women  Hispanic  White  \
CensusId                                                                   
27001     Minnesota    Aitkin     15839    8064    7775       1.1   94.4   
27003     Minnesota     Anoka    338764  169586  169178       3.9   83.8   
27005     Minnesota    Becker     33138   16601   16537       1.8   86.8   
27007     Minnesota  Beltrami     45434   22740   22694       1.9   73.4   
27009     Minnesota    Benton     39221   19500   19721       1.9   92.7   

          Black  Native  Asian  Pacific  Citizen   Income  IncomeErr  \
CensusId                                                               
27001       0.3     1.8    0.3      0.0    13027  42252.0      893.0   
27003       4.9     0.6    4.1      0.0   245735  70873.0      867.0   
27005       0.5     7.0    0.5      0.0    24673  52038.0     1729.0   
27007       0.8    20.0    0.9      0.1    33584  44757.0     2090.0   
27009       2.3     0.3    1.1      0.0    29110  52178.0     2429.0   

          IncomePerCap  IncomePerCapErr  Poverty  ChildPoverty  Professional  \
CensusId                                                                       
27001            25591             1517     12.4          13.7          27.6   
27003            31775              446      7.7          10.4          36.7   
27005            26399              623     12.8          17.0          32.7   
27007            22524              984     18.8          23.6          31.4   
27009            25341              656     14.9          18.7          29.5   

          Service  Office  Construction  Production  Drive  Carpool  Transit  \
CensusId                                                                       
27001        22.5    23.9          13.1        13.0   79.4      9.3      0.3   
27003        15.4    25.1           8.5        14.3   82.1      8.5      3.4   
27005        17.5    20.2          13.7        15.9   77.8     10.8      0.4   
27007        22.9    24.4           9.5        11.7   75.8     11.2      2.0   
27009        17.4    21.3          11.5        20.3   81.9      9.1      1.0   

          Walk  OtherTransp  WorkAtHome  MeanCommute  Employed  PrivateWork  \
CensusId                                                                      
27001      4.3          1.5         5.2         25.1      6428         75.4   
27003      1.1          0.9         4.0         28.1    181190         84.7   
27005      2.9          1.5         6.6         22.8     15929         74.3   
27007      3.2          1.3         6.5         20.2     20579         66.8   
27009      1.5          1.6         4.9         23.3     20104         82.9   

          PublicWork  SelfEmployed  FamilyWork  Unemployment  
CensusId                                                      
27001           15.2           9.2         0.2           7.3  
27003           11.0           4.2         0.1           6.2  
27005           16.4           9.0         0.3           3.5  
27007           26.9           6.2         0.1          10.6  
27009           11.2           5.7         0.2           6.7

In [21]:
dataA[['IncomePerCap','SelfEmployed']].corr()

IncomePerCap  SelfEmployed
IncomePerCap      1.000000     -0.128412
SelfEmployed     -0.128412      1.000000

In [22]:
fig =  dataA.plot(kind = 'scatter', x = 'IncomePerCap', y = 'SelfEmployed')

### 1.D

In [32]:
setA = ['Minnesota']
mask = data['State'].isin(setA)
dataB = data[mask]
dataB.head()

State    County  TotalPop     Men   Women  Hispanic  White  \
CensusId                                                                   
27001     Minnesota    Aitkin     15839    8064    7775       1.1   94.4   
27003     Minnesota     Anoka    338764  169586  169178       3.9   83.8   
27005     Minnesota    Becker     33138   16601   16537       1.8   86.8   
27007     Minnesota  Beltrami     45434   22740   22694       1.9   73.4   
27009     Minnesota    Benton     39221   19500   19721       1.9   92.7   

          Black  Native  Asian  Pacific  Citizen   Income  IncomeErr  \
CensusId                                                               
27001       0.3     1.8    0.3      0.0    13027  42252.0      893.0   
27003       4.9     0.6    4.1      0.0   245735  70873.0      867.0   
27005       0.5     7.0    0.5      0.0    24673  52038.0     1729.0   
27007       0.8    20.0    0.9      0.1    33584  44757.0     2090.0   
27009       2.3     0.3    1.1      0.0    29110  52178.0     2429.0   

          IncomePerCap  IncomePerCapErr  Poverty  ChildPoverty  Professional  \
CensusId                                                                       
27001            25591             1517     12.4          13.7          27.6   
27003            31775              446      7.7          10.4          36.7   
27005            26399              623     12.8          17.0          32.7   
27007            22524              984     18.8          23.6          31.4   
27009            25341              656     14.9          18.7          29.5   

          Service  Office  Construction  Production  Drive  Carpool  Transit  \
CensusId                                                                       
27001        22.5    23.9          13.1        13.0   79.4      9.3      0.3   
27003        15.4    25.1           8.5        14.3   82.1      8.5      3.4   
27005        17.5    20.2          13.7        15.9   77.8     10.8      0.4   
27007        22.9    24.4           9.5        11.7   75.8     11.2      2.0   
27009        17.4    21.3          11.5        20.3   81.9      9.1      1.0   

          Walk  OtherTransp  WorkAtHome  MeanCommute  Employed  PrivateWork  \
CensusId                                                                      
27001      4.3          1.5         5.2         25.1      6428         75.4   
27003      1.1          0.9         4.0         28.1    181190         84.7   
27005      2.9          1.5         6.6         22.8     15929         74.3   
27007      3.2          1.3         6.5         20.2     20579         66.8   
27009      1.5          1.6         4.9         23.3     20104         82.9   

          PublicWork  SelfEmployed  FamilyWork  Unemployment  
CensusId                                                      
27001           15.2           9.2         0.2           7.3  
27003           11.0           4.2         0.1           6.2  
27005           16.4           9.0         0.3           3.5  
27007           26.9           6.2         0.1          10.6  
27009           11.2           5.7         0.2           6.7

In [33]:
fig =  dataB.plot(kind = 'scatter', x = 'IncomePerCap', y = 'SelfEmployed')

In [34]:
dataB[['IncomePerCap','SelfEmployed']].corr()

IncomePerCap  SelfEmployed
IncomePerCap      1.000000     -0.253855
SelfEmployed     -0.253855      1.000000

## Task 2: All you need is love… And a dog!

Here we are going to build a classifier to predict whether an animal from an animal shelter will be adopted or not (aac_intakes_outcomes.csv, available at: https://www.kaggle.com/aaronschlegel/austin-animal-center-shelter-intakes-and-outcomes/version/1#aac_intakes_outcomes.csv). You will be working with the following features:

1. *animal_type:* Type of animal. May be one of 'cat', 'dog', 'bird', etc.
2. *intake_year:* Year of intake
3. *intake_condition:* The intake condition of the animal. Can be one of 'normal', 'injured', 'sick', etc.
4. *intake_number:* The intake number denoting the number of occurrences the animal has been brought into the shelter. Values higher than 1 indicate the animal has been taken into the shelter on more than one occasion.
5. *intake_type:* The type of intake, for example, 'stray', 'owner surrender', etc.
6. *sex_upon_intake:* The gender of the animal and if it has been spayed or neutered at the time of intake
7. *age_upon\_intake_(years):* The age of the animal upon intake represented in years
8. *time_in_shelter_days:* Numeric value denoting the number of days the animal remained at the shelter from intake to outcome.
9. *sex_upon_outcome:* The gender of the animal and if it has been spayed or neutered at time of outcome
10. *age_upon\_outcome_(years):* The age of the animal upon outcome represented in years
11. *outcome_type:* The outcome type. Can be one of ‘adopted’, ‘transferred’, etc.

**A)** Load the dataset and convert categorical features to a suitable numerical representation (use dummy-variable encoding). Split the data into a training set (80%) and a test set (20%). Pair each feature vector with the corresponding label, i.e., whether the outcome_type is adoption or not. Standardize the values of each feature in the data to have mean 0 and variance 1. The use of external libraries is not permitted in part A, except for numpy and pandas.

**B)** Train a logistic regression classifier on your training set. Logistic regression returns probabilities as predictions, so in order to arrive at a binary prediction, you need to put a threshold on the predicted probabilities. For the decision threshold of 0.5, present the performance of your classifier on the test set by displaying the confusion matrix. Based on the confusion matrix, manually calculate accuracy, precision, recall, and F1-score with respect to the positive and the negative class. Vary the value of the threshold in the range from 0 to 1 and visualize the value of accuracy, precision, recall, and F1-score (with respect to both classes) as a function of the threshold. The shelter has a limited capacity and has no other option but to put to sleep animals with a low probability of adoption. What metric (precision, recall, accuracy, or F1-score) and with respect to what class is the most relevant when choosing the threshold in this scenario, and why? Explain your reasoning.

**C)** Reduce the number of features by selecting the subset of the k best features. Use greedy backward selection to iteratively remove features. Evaluate performance and visualize the result using 5-fold cross-validation on the training set as a function of k, where k = 1, 5, 10, 15, 20, 25, 30. Choose the optimal k and justify your choice. Interpret the top-k features and their impact on the probability of adoption.

**D)** Train a random forest. Use 5-fold cross-validation on the training set to fine-tune the parameters of the classifier using a grid search on the number of estimators "n_estimators" and the max depth of the trees "max_depth". For the chosen parameters, estimate the performance of your classifier on the test set by presenting the confusion matrix, accuracy, precision, recall, and F1-score with respect to both classes and compare the performance with the performance of the logistic regression. Interpret the results.

You are allowed to use the scikit-learn library to implement your classifiers.

In [90]:
data  = pd.read_csv(data_folder + 'aac_intakes_outcomes.csv')

In [91]:
data1 = data[['animal_type','intake_year','intake_condition','intake_number',\
              'intake_type','sex_upon_intake','age_upon_intake_(years)',\
              'time_in_shelter_days','sex_upon_outcome','age_upon_outcome_(years)', 'outcome_type']]
data1.head()

animal_type  intake_year intake_condition  intake_number      intake_type  \
0         Dog         2017           Normal            1.0            Stray   
1         Dog         2014           Normal            2.0    Public Assist   
2         Dog         2014           Normal            3.0    Public Assist   
3         Dog         2014           Normal            1.0  Owner Surrender   
4         Dog         2013          Injured            1.0    Public Assist   

  sex_upon_intake  age_upon_intake_(years)  time_in_shelter_days  \
0   Neutered Male                     10.0              0.588194   
1   Neutered Male                      7.0              1.259722   
2   Neutered Male                      6.0              1.113889   
3   Neutered Male                     10.0              4.970139   
4   Neutered Male                     16.0              0.119444   

  sex_upon_outcome  age_upon_outcome_(years)     outcome_type  
0    Neutered Male                      10.0  Return to Owner  
1    Neutered Male                       7.0  Return to Owner  
2    Neutered Male                       6.0  Return to Owner  
3    Neutered Male                      10.0         Transfer  
4    Neutered Male                      16.0  Return to Owner

In [92]:
categoricals = ['animal_type', 'intake_type','sex_upon_outcome','intake_condition','sex_upon_intake']
data_dummies = pd.get_dummies(data1, columns = categoricals)
data_dummies.head()

intake_year  intake_number  age_upon_intake_(years)  time_in_shelter_days  \
0         2017            1.0                     10.0              0.588194   
1         2014            2.0                      7.0              1.259722   
2         2014            3.0                      6.0              1.113889   
3         2014            1.0                     10.0              4.970139   
4         2013            1.0                     16.0              0.119444   

   age_upon_outcome_(years)     outcome_type  animal_type_Bird  \
0                      10.0  Return to Owner                 0   
1                       7.0  Return to Owner                 0   
2                       6.0  Return to Owner                 0   
3                      10.0         Transfer                 0   
4                      16.0  Return to Owner                 0   

   animal_type_Cat  animal_type_Dog  animal_type_Other  \
0                0                1                  0   
1                0                1                  0   
2                0                1                  0   
3                0                1                  0   
4                0                1                  0   

            ...             intake_condition_Normal  intake_condition_Nursing  \
0           ...                                   1                         0   
1           ...                                   1                         0   
2           ...                                   1                         0   
3           ...                                   1                         0   
4           ...                                   0                         0   

   intake_condition_Other  intake_condition_Pregnant  intake_condition_Sick  \
0                       0                          0                      0   
1                       0                          0                      0   
2                       0                          0                      0   
3                       0                          0                      0   
4                       0                          0                      0   

   sex_upon_intake_Intact Female  sex_upon_intake_Intact Male  \
0                              0                            0   
1                              0                            0   
2                              0                            0   
3                              0                            0   
4                              0                            0   

   sex_upon_intake_Neutered Male  sex_upon_intake_Spayed Female  \
0                              1                              0   
1                              1                              0   
2                              1                              0   
3                              1                              0   
4                              1                              0   

   sex_upon_intake_Unknown  
0                        0  
1                        0  
2                        0  
3                        0  
4                        0  

[5 rows x 33 columns]

In [127]:
label = pd.get_dummies(data_dummies, 'outcome_type')
label = label[['outcome_type_Adoption']]


In [122]:
mask = label['outcome_type_Adoption'] == 0
training  = pd.concat([data_dummies[mask].sample(frac = 0.8, replace = False),\
           data_dummies[~mask].sample(frac = 0.8, replace = False)], axis = 0)\
            .drop(columns = 'outcome_type')
label_train = label[label.index.isin(training.index)]

In [123]:
testing  = data_dummies[~data_dummies.index.isin(training.index)].drop(columns = 'outcome_type')
label_test = label[label.index.isin(testing.index)]

In [120]:
def standardize(col):
    return (col - np.mean(col)) / np.std(col)

In [133]:
training_n = training.apply(lambda x : standardize(x))
training_n.head()

intake_year  intake_number  age_upon_intake_(years)  \
64510     1.200378       -0.27971                 0.650730   
67192     1.200378       -0.27971                -0.708474   
76329     1.200378       -0.27971                -0.614964   
40933     0.432513       -0.27971                 0.995490   
125       0.432513       -0.27971                 4.098331   

       time_in_shelter_days  age_upon_outcome_(years)  animal_type_Bird  \
64510             -0.403430                  0.637299         -0.065587   
67192             -0.401971                 -0.718779         -0.065587   
76329             -0.312130                 -0.625484         -0.065587   
40933             -0.190073                  0.981265         -0.065587   
125               -0.383993                  4.076966         -0.065587   

       animal_type_Cat  animal_type_Dog  animal_type_Other  \
64510        -0.766545        -1.152589           4.142305   
67192         1.304556        -1.152589          -0.241411   
76329         1.304556        -1.152589          -0.241411   
40933        -0.766545         0.867612          -0.241411   
125          -0.766545         0.867612          -0.241411   

       intake_type_Euthanasia Request           ...             \
64510                       -0.055254           ...              
67192                       -0.055254           ...              
76329                       -0.055254           ...              
40933                       -0.055254           ...              
125                         -0.055254           ...              

       intake_condition_Normal  intake_condition_Nursing  \
64510                -2.699685                 -0.157248   
67192                 0.370414                 -0.157248   
76329                 0.370414                 -0.157248   
40933                 0.370414                 -0.157248   
125                   0.370414                 -0.157248   

       intake_condition_Other  intake_condition_Pregnant  \
64510               -0.042515                  -0.025371   
67192               -0.042515                  -0.025371   
76329               -0.042515                  -0.025371   
40933               -0.042515                  -0.025371   
125                 -0.042515                  -0.025371   

       intake_condition_Sick  sex_upon_intake_Intact Female  \
64510              -0.200618                      -0.650716   
67192              -0.200618                      -0.650716   
76329              -0.200618                      -0.650716   
40933              -0.200618                      -0.650716   
125                -0.200618                      -0.650716   

       sex_upon_intake_Intact Male  sex_upon_intake_Neutered Male  \
64510                     1.463530                      -0.435214   
67192                    -0.683279                      -0.435214   
76329                    -0.683279                      -0.435214   
40933                    -0.683279                       2.297718   
125                      -0.683279                      -0.435214   

       sex_upon_intake_Spayed Female  sex_upon_intake_Unknown  
64510                      -0.404204                -0.303907  
67192                      -0.404204                 3.290477  
76329                      -0.404204                 3.290477  
40933                      -0.404204                -0.303907  
125                         2.473996                -0.303907  

[5 rows x 32 columns]

In [138]:
model  = LogisticRegression().fit(training_n, label_train.values.ravel())

In [139]:
training_n.head()

intake_year  intake_number  age_upon_intake_(years)  \
64510     1.200378       -0.27971                 0.650730   
67192     1.200378       -0.27971                -0.708474   
76329     1.200378       -0.27971                -0.614964   
40933     0.432513       -0.27971                 0.995490   
125       0.432513       -0.27971                 4.098331   

       time_in_shelter_days  age_upon_outcome_(years)  animal_type_Bird  \
64510             -0.403430                  0.637299         -0.065587   
67192             -0.401971                 -0.718779         -0.065587   
76329             -0.312130                 -0.625484         -0.065587   
40933             -0.190073                  0.981265         -0.065587   
125               -0.383993                  4.076966         -0.065587   

       animal_type_Cat  animal_type_Dog  animal_type_Other  \
64510        -0.766545        -1.152589           4.142305   
67192         1.304556        -1.152589          -0.241411   
76329         1.304556        -1.152589          -0.241411   
40933        -0.766545         0.867612          -0.241411   
125          -0.766545         0.867612          -0.241411   

       intake_type_Euthanasia Request           ...             \
64510                       -0.055254           ...              
67192                       -0.055254           ...              
76329                       -0.055254           ...              
40933                       -0.055254           ...              
125                         -0.055254           ...              

       intake_condition_Normal  intake_condition_Nursing  \
64510                -2.699685                 -0.157248   
67192                 0.370414                 -0.157248   
76329                 0.370414                 -0.157248   
40933                 0.370414                 -0.157248   
125                   0.370414                 -0.157248   

       intake_condition_Other  intake_condition_Pregnant  \
64510               -0.042515                  -0.025371   
67192               -0.042515                  -0.025371   
76329               -0.042515                  -0.025371   
40933               -0.042515                  -0.025371   
125                 -0.042515                  -0.025371   

       intake_condition_Sick  sex_upon_intake_Intact Female  \
64510              -0.200618                      -0.650716   
67192              -0.200618                      -0.650716   
76329              -0.200618                      -0.650716   
40933              -0.200618                      -0.650716   
125                -0.200618                      -0.650716   

       sex_upon_intake_Intact Male  sex_upon_intake_Neutered Male  \
64510                     1.463530                      -0.435214   
67192                    -0.683279                      -0.435214   
76329                    -0.683279                      -0.435214   
40933                    -0.683279                       2.297718   
125                      -0.683279                      -0.435214   

       sex_upon_intake_Spayed Female  sex_upon_intake_Unknown  
64510                      -0.404204                -0.303907  
67192                      -0.404204                 3.290477  
76329                      -0.404204                 3.290477  
40933                      -0.404204                -0.303907  
125                         2.473996                -0.303907  

[5 rows x 32 columns]

In [140]:
testing_n = testing.sub(training.mean()).div(training.std())

In [141]:
proba = model.predict_proba(testing_n)
first =[item[0] for item in proba]
                        

In [144]:
less = [item[0] for item in proba if item[0]<0.5]

In [145]:
less

[0.3649028687677005,
 0.4823319333684677,
 0.49302303645601364,
 0.4621791650933855,
 0.4023807508377917,
 0.4961549391889216,
 0.49509379324288716,
 0.4994184846480566,
 0.4728987619822651,
 0.415808768457241,
 0.49834663301552784,
 0.4364571742915988,
 0.4268324283519108,
 0.4766854699192853,
 0.4939559595961964,
 0.46482778951410075,
 0.4945511638337,
 0.47786202697665137,
 0.4221898226969899,
 0.4805818838103828,
 0.4794295627911723,
 0.49522947193076405,
 0.478528814730412,
 0.4988273815456984,
 0.467700759450674,
 0.4944085835594675,
 0.49219282367532335,
 0.4990592396397039,
 0.4958668145199927,
 0.4984289003536668,
 0.49515853220691763,
 0.49481092624312895]

In [89]:
testing_n.shape

(15935, 32)

In [150]:
a = [1,2,3]
a[0]

1

In [147]:
a = np.array([1, 2, 3])
b = np.array([2, 3, 4])
np.hstack((a,b))


array([1, 2, 3, 2, 3, 4])

**D)** Train a random forest. Use 5-fold cross-validation on the training set to fine-tune the parameters of the classifier using a grid search on the number of estimators "n_estimators" and the max depth of the trees "max_depth". For the chosen parameters, estimate the performance of your classifier on the test set by presenting the confusion matrix, accuracy, precision, recall, and F1-score with respect to both classes and compare the performance with the performance of the logistic regression. Interpret the results.